In [13]:
from iFinDPy import *
import pandas as pd
import numpy as np
import tqdm
import datetime
import sqlite3

from iFinDPy import *

from const import *


# 数据下载

In [6]:
userID = "zxqh172"
password = "OiLFuJE95"

In [7]:
THS_iFinDLogin(userID,password)

0

In [40]:
PRICE_COLUMNS = 'open,high,low,close,avgPrice,changeRatio,volume,amount'


In [74]:
PATH_DATABASES = "databases/{}"

PATH_PRICE_BYX = PATH_DATABASES.format("price_byx.csv")
PATH_PRICE_YY = PATH_DATABASES.format("price_yy.csv")
PATH_PRICE_SL = PATH_DATABASES.format("price_sl.csv")

PATH_INDEX_SP = PATH_DATABASES.format("index_sp.csv")
PATH_INDEX_BYX = PATH_DATABASES.format("index_byx.csv")
PATH_INDEX_SYHG = PATH_DATABASES.format("index_syhg.csv")

PATH_AU = PATH_DATABASES.format("au_price.csv")

PATH_DICT = {"EBZL.DCE": PATH_PRICE_BYX, "SCZL.SHF": PATH_PRICE_YY, "LZL.DCE": PATH_PRICE_SL,
             "NHCI.SL": PATH_INDEX_SP, "NHEBI.SL": PATH_INDEX_BYX, "NHPCI.SL": PATH_INDEX_SYHG,
             "AU9999.SHG": PATH_AU}

In [78]:
class DataDownloader(object):
    """
    used to downloaded the data
    """
    path_dict = PATH_DICT

    def  save_data(self, data: pd.DataFrame, path):
        data.to_csv(path)
        pass

    def download_data(self, index, 
                      columns=PRICE_COLUMNS, 
                      start_date=START_DATE, end_date=END_DATE):
        
        data = THS_HQ(index, columns, 'CPS:7', start_date, end_date)
        return data.data
    
    def download_all_data(self):
        for index, path in tqdm.tqdm(self.path_dict.items()):
            # print(index)
            data = self.download_data(index=index)
            data.to_csv(path)
        pass
        


In [76]:
data_downloader = DataDownloader()
data_downloader.download_all_data()

# read data

In [103]:
class DataReader(object):
    """
    used to read the data
    """
    def __init__(self) -> None:
        self.trade_date_list = self.get_trade_date_list()
        pass

    def get_trade_date_list(self):
        data_yx = pd.read_csv(PATH_PRICE_BYX)
        return pd.DataFrame(data_yx["time"])
    
    def _read_data(self, path):
        data = pd.read_csv(path)
        data_ = pd.merge(data, pd.DataFrame(columns=["time"], data=self.trade_date_list), how="right", on="time")
        data_.drop(columns=["Unnamed: 0"], inplace=True)
        data_.index = pd.to_datetime(data_["time"])
        return data_
    

    def read_price_byx(self):
        return self._read_data(path=PATH_PRICE_BYX)
    
    def read_price_yy(self):
        return self._read_data(path=PATH_PRICE_YY)
    
    def read_price_sl(self):
        return self._read_data(path=PATH_PRICE_SL)
    
    def read_index_sp(self):
        return self._read_data(path=PATH_INDEX_SP)
    
    def read_au(self):
        return self._read_data(path=PATH_AU)
    
    

In [105]:
data_reader = DataReader()
traed_date_list = data_reader.get_trade_date_list()

In [106]:
data_reader.read_au()

,time,thscode,open,high,low,close,avgPrice,changeRatio,volume,amount
time,,,,,,,,,,
2019-09-26,2019-09-26,AU9999.SHG,354.00,354.00,347.15,347.20,347758.961754,-1.4057,17575.60,6.112072e+09
2019-09-27,2019-09-27,AU9999.SHG,346.02,349.50,344.44,345.25,340334.962448,-0.5616,14990.02,5.101628e+09
2019-09-30,2019-09-30,AU9999.SHG,343.33,347.99,342.03,344.49,345420.504487,-0.2201,12293.28,4.246351e+09
2019-10-08,2019-10-08,AU9999.SHG,344.33,344.46,342.60,344.00,327388.421784,-0.1422,11608.30,3.800423e+09
2019-10-09,2019-10-09,AU9999.SHG,349.00,349.00,344.10,348.39,349779.213802,1.2762,16046.08,5.612585e+09
...,...,...,...,...,...,...,...,...,...,...
2024-04-19,2024-04-19,AU9999.SHG,562.80,573.96,561.00,565.55,567205.309924,0.3870,9604.92,5.447962e+09
2024-04-22,2024-04-22,AU9999.SHG,563.10,567.60,555.80,556.55,560071.753174,-1.5914,11469.94,6.423989e+09
2024-04-23,2024-04-23,AU9999.SHG,552.80,553.55,542.01,544.01,545057.103454,-2.2532,11876.42,6.473327e+09


In [91]:
df_test = pd.read_csv(PATH_INDEX_SYHG)

In [92]:
df_test.columns

Index(['Unnamed: 0', 'time', 'thscode', 'open', 'high', 'low', 'close',
       'avgPrice', 'changeRatio', 'volume', 'amount'],
      dtype='object')

In [87]:
pd.merge(df_test, pd.DataFrame(columns=["time"], data=traed_date_list), how="right", on="time").columns

Index(['Unnamed: 0', 'time', 'thscode', 'open', 'high', 'low', 'close',
       'avgPrice', 'changeRatio', 'volume', 'amount'],
      dtype='object')

In [70]:
df_yx = THS_HQ('EBZL.DCL','open,high,low,close,avgPrice,changeRatio,volume,amount','CPS:7','2017-01-01','2024-04-25')

In [72]:
df_yx

errorcode=-4210
errmsg=error happen with input parameters, please have a check.
data=None

In [62]:
pd.DataFrame(df_yx)

ValueError: DataFrame constructor not properly called!

In [73]:
THS_HQ('EBZL.DCE','open,high,low,close,avgPrice,changeRatio,volume,amount','CPS:7','2017-01-01','2024-04-25')

errorcode=0
errmsg=
data=            time   thscode  open  high   low  close     avgPrice  changeRatio  \
0     2019-09-26  EBZL.DCE  8100  8216  8051   8125  8145.188684     1.562500   
1     2019-09-27  EBZL.DCE  8020  8068  7901   7940  7977.451471    -1.610905   
2     2019-09-30  EBZL.DCE  7905  7999  7905   7976  7956.888921    -0.012536   
3     2019-10-08  EBZL.DCE  7950  7998  7920   7968  7962.263254     0.150830   
4     2019-10-09  EBZL.DCE  7971  8018  7940   7952  7974.696077    -0.125597   
...          ...       ...   ...   ...   ...    ...          ...          ...   
1104  2024-04-19  EBZL.DCE  9652  9780  9580   9652  9668.180248     0.489328   
1105  2024-04-22  EBZL.DCE  9605  9685  9528   9531  9613.377602    -1.417046   
1106  2024-04-23  EBZL.DCE  9460  9525  9405   9415  9473.398696    -1.423935   
1107  2024-04-24  EBZL.DCE  9386  9535  9386   9498  9457.560278     0.263908   
1108  2024-04-25  EBZL.DCE  9498  9548  9455   9499  9494.179310     0.444115   

  

In [ ]:
THS_HQ('EBZL.DCE','open,high,low,close,avgPrice,changeRatio,volume,amount','CPS:7','2017-01-01','2024-04-25')